In [1]:
import pymssql
import pyodbc
import json
import os

C:\Users\yil4005\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [2]:
JSON_FILE = os.path.expanduser("C:/Users/yil400/\Desktop/WCM/TA_DataManagement/SQLtoJSON.json")

In [3]:
connection = pyodbc.connect('Driver={SQL Server};''Server=localhost\SQLEXPRESS;''Database=prj0111;''Trusted_Connection=yes;')

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (67)')

In [12]:
cur = connection.cursor()

In [13]:
cur.execute("select person_id from [CUMC\yuh4003].person;")

In [14]:
resultlist_patient = cur.fetchall()

In [16]:
print(resultlist_patient[0:10])

[(2, ), (3, ), (4, ), (5, ), (6, ), (7, ), (8, ), (9, ), (10, ), (11, )]


In [20]:
type(resultlist_patient)
type(resultlist_patient[0])

list

In [26]:
cur.execute(
"""select Person.person_id,gender_source_value,race_source_value,year_of_birth,
    care_site.care_site_name 
    from [CUMC\yuh4003].Person 
    left join [CUMC\yil4005].care_site 
    on person.care_site_id=care_site.care_site_id""")
resultlist_demo = cur.fetchall()

In [27]:
cur.execute(
"""select person_id,visit_occurrence_id, procedure_date,procedure_concept_id,provider_id 
    from [CUMC\yuh4003].procedure_occurrence""")
resultlist_ce = cur.fetchall()

In [29]:
cur.execute(
"""select person_id,visit_occurrence_id,visit_start_date,visit_end_date,visit_concept_id, care_site_id 
from [CUMC\yuh4003].visit_occurrence""")
resultlist_ap = cur.fetchall()

In [31]:
cur.execute(
"""select co.person_id,co.visit_occurrence_id,condition_start_date, condition_concept_id 
    from [CUMC\yil4005].condition_occurrence co""")
resultlist_cd = cur.fetchall()

In [32]:
cur.execute(
"""select person_id,visit_occurrence_id,drug_exposure_start_date,drug_exposure_end_date,drug_concept_id,provider_id 
    from [CUMC\yil4005].drug_exposure de""")
resultlist_drug = cur.fetchall()

Python "class" is a way to bundle data and functionality together. It is a type of object, allowing new instances of that type to be made.

In [5]:
class Patient:
    def __init__(self, parameters): #a way to instantiate the class; self is the instance of the class
        self.acct = parameters[0]

In [6]:
class Demographics:
    def __init__(self, parameters):
        self.acct = parameters[0]
        self.sex = parameters[1]
        self.race = parameters[2]
        self.age = parameters[3]
        self.caresite = parameters[4]

In [7]:
class Appointment:
    def __init__(self, parameters):
        self.acct = parameters[0]
        self.visitid = parameters[1]
        self.date = str(parameters[2])
        self.enddate = str(parameters[3])
        self.type = parameters[4]
        self.site = parameters[5]

In [8]:
class Clinical_event:
    def __init__(self, parameters):
        self.acct = parameters[0]
        self.visitid = parameters[1]
        self.date = str(parameters[2])
        self.event = parameters[3]
        self.providerid = parameters[4]

In [9]:
class Diagnosis:
    def __init__(self, parameters):
        self.acct = parameters[0]
        self.visitid = parameters[1]
        self.date = str(parameters[2])
        self.diag = parameters[3]

In [10]:
class Drug:
    def __init__(self, parameters):
        self.acct = parameters[0]
        self.visitid = parameters[1]
        self.startdate = str(parameters[2])
        self.enddate = str(parameters[3])
        self.drug = parameters[4]
        self.providerid = parameters[5]

In [51]:
CE={}

In [52]:
for res in resultlist_patient:
    p = Patient(res)
    CE[p.acct] = dict()
    CE[p.acct]['appt'] = dict()
    CE[p.acct]['sex'] = 'NA'
    CE[p.acct]['race'] = 'NA'
    CE[p.acct]['care_site'] = 'NA'
    CE[p.acct]['YOB'] = -1
    CE[p.acct]['visitIDlist'] = []

In [53]:
print(CE[2])

{'appt': {}, 'visitIDlist': [], 'YOB': -1, 'race': 'NA', 'care_site': 'NA', 'sex': 'NA'}


In [54]:
for res in resultlist_demo:
    demo = Demographics(res)
    try:
        CE[demo.acct]['sex'] = demo.sex
        CE[demo.acct]['YOB'] = int(demo.age)
        CE[demo.acct]['race'] = demo.race
        CE[demo.acct]['care_site'] = demo.caresite
    except:
        pass

In [55]:
print(CE[2])

{'appt': {}, 'visitIDlist': [], 'YOB': 1943, 'race': 1, 'care_site': None, 'sex': 1}


In [56]:
for res in resultlist_ap:
    ap = Appointment(res)
    if ap.acct in CE:
        CE[ap.acct]['appt'][ap.date] = dict()
        CE[ap.acct]['appt'][ap.date]['ID'] = ap.visitid
        CE[ap.acct]['appt'][ap.date]['actualdate'] = ap.date
        CE[ap.acct]['appt'][ap.date]['enddate'] = ap.enddate
        CE[ap.acct]['appt'][ap.date]['type'] = ap.type
        CE[ap.acct]['appt'][ap.date]['diag'] = list()
        CE[ap.acct]['appt'][ap.date]['drug'] = list()
        CE[ap.acct]['appt'][ap.date]['proc'] = list()             

In [57]:
CE[2]

{'YOB': 1943,
 'appt': {'2008-10-04 00:00:00.0000000': {'ID': 32,
   'actualdate': '2008-10-04 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-04 00:00:00.0000000',
   'proc': [],
   'type': 0},
  '2008-10-08 00:00:00.0000000': {'ID': 45,
   'actualdate': '2008-10-08 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-08 00:00:00.0000000',
   'proc': [],
   'type': 0},
  '2008-10-09 00:00:00.0000000': {'ID': 43,
   'actualdate': '2008-10-09 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-18 00:00:00.0000000',
   'proc': [],
   'type': 0},
  '2008-10-20 00:00:00.0000000': {'ID': 51,
   'actualdate': '2008-10-20 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-20 00:00:00.0000000',
   'proc': [],
   'type': 0},
  '2008-12-09 00:00:00.0000000': {'ID': 25,
   'actualdate': '2008-12-09 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-12-09 00:00:00.0000000',
   'proc': [],
   'type

In [58]:
for res in resultlist_ce:
    ce = Clinical_event(res)
    if ce.acct in CE:
        for date in CE[ce.acct]['appt']:
            if ce.visitid == CE[ce.acct]['appt'][date]['ID']:
                CE[ce.acct]['appt'][date]['proc'].append(ce.event)
                CE[ce.acct]['appt'][date]['proc'] = sorted(CE[ce.acct]['appt'][date]['proc'])

In [59]:
CE[2]

{'YOB': 1943,
 'appt': {'2008-10-04 00:00:00.0000000': {'ID': 32,
   'actualdate': '2008-10-04 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-04 00:00:00.0000000',
   'proc': [2105893],
   'type': 0},
  '2008-10-08 00:00:00.0000000': {'ID': 45,
   'actualdate': '2008-10-08 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-08 00:00:00.0000000',
   'proc': [2211359, 2211359, 2211740],
   'type': 0},
  '2008-10-09 00:00:00.0000000': {'ID': 43,
   'actualdate': '2008-10-09 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-18 00:00:00.0000000',
   'proc': [2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2102912,
    2211361,
    2211733,
    2314213],
   'type': 0},
  '2008-10-20 00:00:00.0000000': {'ID': 51,
   'actualdate': '2008-10-20 00:00:00.0000000',
   'diag': [],
   'drug': [],
   'enddate': '2008-10-20 00:00:00.0000000',
   'proc': [2005421, 2514407],
   'type': 0},
  '20

In [60]:
for res in resultlist_cd:
    cd = Diagnosis(res)
    if cd.acct in CE:
        for date in CE[cd.acct]['appt']:
            if cd.visitid == CE[cd.acct]['appt'][date]['ID']:
                CE[cd.acct]['appt'][date]['diag'].append(cd.diag)
                CE[cd.acct]['appt'][date]['diag'] = sorted(CE[cd.acct]['appt'][date]['diag'])

In [61]:
CE[2]

{'YOB': 1943,
 'appt': {'2008-10-04 00:00:00.0000000': {'ID': 32,
   'actualdate': '2008-10-04 00:00:00.0000000',
   'diag': [434502, 434502, 439947, 43531028],
   'drug': [],
   'enddate': '2008-10-04 00:00:00.0000000',
   'proc': [2105893],
   'type': 0},
  '2008-10-08 00:00:00.0000000': {'ID': 45,
   'actualdate': '2008-10-08 00:00:00.0000000',
   'diag': [27674, 200219, 200219, 200219, 200219, 317898, 40490986],
   'drug': [],
   'enddate': '2008-10-08 00:00:00.0000000',
   'proc': [2211359, 2211359, 2211740],
   'type': 0},
  '2008-10-09 00:00:00.0000000': {'ID': 43,
   'actualdate': '2008-10-09 00:00:00.0000000',
   'diag': [140673, 201837, 439777, 442077],
   'drug': [],
   'enddate': '2008-10-18 00:00:00.0000000',
   'proc': [2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2102912,
    2211361,
    2211733,
    2314213],
   'type': 0},
  '2008-10-20 00:00:00.0000000': {'ID': 51,
   'actualdate': '2008-10-20 00:00:00.0000000',
   'diag'

In [64]:
for res in resultlist_drug:
    cdrg = Drug(res)
    if cdrg.acct in CE:
        for date in CE[cdrg.acct]['appt']:
            if cdrg.visitid == CE[cdrg.acct]['appt'][date]['ID']:
                d = dict()
                d['name'] = cdrg.drug
                d['startdate'] = cdrg.startdate
                CE[cdrg.acct]['appt'][date]['drug'].append(d)

In [66]:
CE[2]

{'YOB': 1943,
 'appt': {'2008-10-04 00:00:00.0000000': {'ID': 32,
   'actualdate': '2008-10-04 00:00:00.0000000',
   'diag': [434502, 434502, 439947, 43531028],
   'drug': [],
   'enddate': '2008-10-04 00:00:00.0000000',
   'proc': [2105893],
   'type': 0},
  '2008-10-08 00:00:00.0000000': {'ID': 45,
   'actualdate': '2008-10-08 00:00:00.0000000',
   'diag': [27674, 200219, 200219, 200219, 200219, 317898, 40490986],
   'drug': [],
   'enddate': '2008-10-08 00:00:00.0000000',
   'proc': [2211359, 2211359, 2211740],
   'type': 0},
  '2008-10-09 00:00:00.0000000': {'ID': 43,
   'actualdate': '2008-10-09 00:00:00.0000000',
   'diag': [140673, 201837, 439777, 442077],
   'drug': [{'enddate': '2008-10-09', 'name': 40227542},
    {'name': 40227542, 'startdate': '2008-10-09'}],
   'enddate': '2008-10-18 00:00:00.0000000',
   'proc': [2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2001172,
    2102912,
    2211361,
    2211733,
    2314213],
   'type': 0},
  '2008

In [44]:
with open(JSON_FILE, 'w') as outfile:
    json.dump(CE, outfile, indent=2, sort_keys=True, separators=(',', ': '))